# Draft

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Settings

In [2]:
# Bitcoin/Thugz League

TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 7/8, 
    'assists': 7/8, 
    'plus_minus': 3/4, 
    'powerplay_points': 3/4, 
    'shots_on_goal': 1, 
    'hits': 1, 
    'blocks': 1
}

GOALIE_CATEGORIES = {
    "wins" : 1, 
    "save_percentage": 7/8, 
    "saves": 3/4, 
    "shutouts": 1/2
}

In [ ]:
# Yahoo Public Leagues

TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 7/8, 
    'assists': 1, 
    'plus_minus': 3/4, 
    'powerplay_points': 3/4, 
    'shots_on_goal': 1, 
    'hits': 1
}

GOALIE_CATEGORIES = {
    "wins" : 1, 
    "goals_against_average": 7/8,
    "save_percentage": 7/8, 
    "shutouts": 1/2
}

### Players

In [3]:
pic = pd.read_csv("../data/adp-yahoo_com.csv")
cap = pd.read_csv("../data/info-capfriendly_com.csv")

# merge
players = pd.merge(pic, cap, how="left", on="name")

# extra features
players["apv"] = players["pick"] 
players["pick"] = players["pick"].rank(method='average', ascending=True)
players['round'] = (players['pick'] // TEAMS) + 1

# organize
players = players[['team', 'age', 'name', 'position', 'apv', 'pick', 'round']]

# peek
players.head()

,team,age,name,position,apv,pick,round
0,Edm,23.0,Connor McDavid,C,1.6,1.0,1.0
1,Col,25.0,Nathan MacKinnon,C,2.5,2.0,1.0
2,Edm,25.0,Leon Draisaitl,"C,LW",3.6,3.0,1.0
3,NYR,29.0,Artemi Panarin,LW,5.2,4.0,1.0
4,Was,35.0,Alex Ovechkin,LW,5.5,5.0,1.0


### Multiple Positions

In [4]:
multi = (
    players
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

multi.head()

,name,position
0,Connor McDavid,C
1,Nathan MacKinnon,C
2,Leon Draisaitl,C
3,Leon Draisaitl,LW
4,Artemi Panarin,LW


### Projection Data

In [5]:
proj = pd.read_csv("../data/projections-generated.csv")

### Goalies

In [6]:
goalies = proj[proj["position"] == "G"].copy()

goalies = (
    goalies
    [["name"] + list(GOALIE_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

for key, value in GOALIE_CATEGORIES.items():
    goalies[key] *= value
    
goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= sum(GOALIE_CATEGORIES.values())
goalies["rollup"] *= 100

goalies = goalies.reset_index()
goalies = goalies[["name", "rollup"]]

goalies.head()

,name,rollup
0,Andrei Vasilevskiy,96.703958
1,Connor Hellebuyck,84.600726
2,Tuukka Rask,77.123548
3,Carter Hart,73.596899
4,Robin Lehner,77.936073


### Skaters

In [7]:
skaters = proj[proj["position"] != "G"].copy()
skaters = (
    skaters
    [["name"] + list(SKATER_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)
     
for key, value in SKATER_CATEGORIES.items():
    skaters[key] *= value
    
skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= sum(SKATER_CATEGORIES.values())
skaters["rollup"] *= 100

skaters = skaters.reset_index()
skaters = skaters[["name", "rollup"]]

skaters.head()

,name,rollup
0,Connor McDavid,59.357591
1,Nathan MacKinnon,62.665760
2,Leon Draisaitl,55.898434
3,Artemi Panarin,53.453218
4,Alex Ovechkin,59.017776


### Merge

In [8]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
rollup.head()

,name,position,rollup
7,Andrei Vasilevskiy,G,96.703958
18,Connor Hellebuyck,G,84.600726
39,Robin Lehner,G,77.936073
36,Tuukka Rask,G,77.123548
62,Frederik Andersen,G,75.055113


### VORP

In [9]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in ROSTER.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * TEAMS)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head()

,name,position,rollup,vorp
7,Andrei Vasilevskiy,G,96.703958,29.603847
18,Connor Hellebuyck,G,84.600726,17.500616
39,Robin Lehner,G,77.936073,10.835963
36,Tuukka Rask,G,77.123548,10.023437
62,Frederik Andersen,G,75.055113,7.955002


### Value Over Next Round

In [10]:
vnr = pd.merge(vorp, players[["name", "round"]], how="inner", on="name")

vnr = (
    vnr
    .sort_values(["rollup"], ascending=[False])
    .groupby(["round", "position"])
    .head(1)
    .sort_values(["position", "round"])
)

vnr["round"] = vnr["round"] - 1
vnr["next"] = vnr["rollup"]
vnr = vnr[["round", "position", "next"]]

### Draft

In [11]:
draft = pd.merge(players, vorp, how="left", on="name", suffixes=("_yahoo", ""))
draft = pd.merge(draft, vnr, how="left", on=["position", "round"])

draft = draft.sort_values("vorp", ascending=False)
draft = draft.groupby("name").head(1)

draft["vorn"] = draft["rollup"] - draft["next"]
draft['rank'] = draft['vorp'].rank(method='average', ascending=False)
draft["arbitrage"] = draft["pick"] - draft["rank"]

draft = draft[[
    'team', 'age', 'name', 'position_yahoo', 
    'rollup', 'vorp', 'vorn',
    'round', 'pick', 'rank', 'arbitrage'
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)
draft["vorn"] = draft["vorn"].round(1)

draft.head()

,team,age,name,position_yahoo,rollup,vorp,vorn,round,pick,rank,arbitrage
7,TB,26.0,Andrei Vasilevskiy,G,96.7,29.6,12.1,1.0,7.0,1.0,6.0
18,Wpg,27.0,Connor Hellebuyck,G,84.6,17.5,6.7,2.0,17.0,2.0,15.0
1,Col,25.0,Nathan MacKinnon,C,62.7,14.2,5.9,1.0,2.0,3.0,-1.0
5,Was,35.0,Alex Ovechkin,LW,59.0,12.4,7.8,1.0,5.0,4.0,1.0
33,Nsh,30.0,Roman Josi,D,54.1,12.4,4.1,3.0,29.0,5.0,24.0


### Targets

In [12]:
draft["target"] = (
    (draft["age"] <= 30) &
    (draft["vorp"] >= -1) &
    (draft["arbitrage"] >= -12) & 
    (draft["vorn"] >= 0)
)

In [13]:
draft.to_csv("../data/draft-thugz_league.csv", index=False)